Project Name: Dog Breed Classification using Convolutional Neural Networks

Libraries Used:
numpy and pandas for data manipulation
matplotlib for data visualization
tqdm for progress tracking
keras for building and training neural networks
sklearn for preprocessing data and evaluating models

Dataset:
The dataset used is the "Dog Breed Identification" dataset, consisting of images of various dog breeds.
It contains a total of 10,222 images belonging to 120 dog breeds.
For this project, a subset of five dog breeds was selected: Scottish Deerhound, Maltese Dog, Afghan Hound, Entlebucher, and Bernese Mountain Dog.

Model Implementation:
Baseline Model: A simple convolutional neural network (CNN) model was implemented without any optimization techniques.
Optimized Model: An optimized CNN model was implemented using three regularization techniques: L1 regularization, L2 regularization, and a combination of both.

Training and Evaluation:
The dataset was split into training, validation, and test sets.
Both models were trained using the training set and evaluated on the validation and test sets.
Model performance metrics such as accuracy were recorded during training and evaluated after training completion.


In [ ]:
# Importing library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.preprocessing import image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.regularizers import l1, l2
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator


## Loading the labels data into dataframe and viewing the data.

In [ ]:
# Read the labels.csv file and check shape and records
labels_all = pd.read_csv('./dogbreedidfromcomp/labels.csv')
print(labels_all.shape)
labels_all.head()

In [ ]:
# Loading number or each breed
breed_all = labels_all['breed']
breed_count = breed_all.value_counts()
breed_count.head()

In [ ]:
# Selecting all breeds because i have high computation power
CLASS_NAME = ['scottish_deerhound', 'maltese_dog', 'afghan_hound', 'entlebucher', 'bernese_mountain_dog']
labels = labels_all[(labels_all['breed'].isin(CLASS_NAME))]
labels = labels.reset_index()
labels.head()

In [ ]:
# Creating numpy matrix with zeros
X_data = np.zeros((len(labels), 224, 224, 3), dtype='float32')
# One hot encoding
Y_data = label_binarize(labels['breed'], classes = CLASS_NAME)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# Reading and converting image to numpy array and normalizing dataset
for i in tqdm(range(len(labels))):
    img = image.load_img('./dogbreedidfromcomp/train/%s.jpg' % labels['id'][i], target_size=(224, 224))
    img = image.img_to_array(img)
    x = np.expand_dims(img.copy(), axis=0)
    X_data[i] = x / 255.0

# Printing train image and one hot encode shape & size
print('\nTrain Images shape: ',X_data.shape,' size: {:,}'.format(X_data.size))
print('One-hot encoded output shape: ',Y_data.shape,' size: {:,}'.format(Y_data.size))

#### Next we will create a network architecture for the model. We have used different types of layers according to their features namely Conv_2d (It is used to create a convolutional kernel that is convolved with the input layer to produce the output tensor), max_pooling2d (It is a downsampling technique which takes out the maximum value over the window defined by poolsize), flatten (It flattens the input and creates a 1D output), Dense (Dense layer produce the output as the dot product of input and kernel).

In [ ]:
# Building the Model
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu', input_shape = (224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 16, kernel_size = (7,7), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 8, kernel_size = (5,5), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(64, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(len(CLASS_NAME), activation = "softmax"))

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001),metrics=['accuracy'])

model.summary()

After defining the network architecture we found out the total parameters as 162,619.

## After defining the network architecture we will start with splitting the test and train data then dividing train data in train and validation data. 

In [ ]:
# Splitting the data set into training and testing data sets
X_train_and_val, X_test, Y_train_and_val, Y_test = train_test_split(X_data, Y_data, test_size = 0.1)
# Splitting the training data set into training and validation data sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train_and_val, Y_train_and_val, test_size = 0.2)

In [ ]:
# Training the model
epochs = 20
batch_size = 84

history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, Y_val))

## Here we analyse how the model is learning with each epoch in terms of accuracy.

In [ ]:
# Plot the training history
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])

plt.show()

## We will use predict function to make predictions using this model also we are finding out the accuracy on the test set.

In [ ]:
Y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

In [ ]:
# Plotting image to compare
plt.imshow(X_test[1,:,:,:])
plt.show()

# Finding max value from predition list and comaparing original value vs predicted
print("Originally : ",labels['breed'][np.argmax(Y_test[1])])
print("Predicted : ",labels['breed'][np.argmax(Y_pred[1])])

In [ ]:
# Save the first model
model.save('saved_models/model.h5')

# Conclusion

We started with downloading the dataset creating the model and finding out the predictions using the model. We can optimize different hyper parameters in order to tune this model for a higher accuracy. This model can be used to predict different breeds of dogs which can be further used by different NGO's working on saving animals and for educational purposes also.

In [ ]:
# Building the Model with Regularization and Dropout
model_optimized = Sequential()

model_optimized.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu', input_shape=(224,224,3), kernel_regularizer=l1(0.01)))
model_optimized.add(MaxPool2D(pool_size=(2,2)))

model_optimized.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', kernel_regularizer=l2(0.01)))
model_optimized.add(MaxPool2D(pool_size=(2,2)))

model_optimized.add(Conv2D(filters=16, kernel_size=(7,7), activation='relu', kernel_regularizer=l1(0.01)))
model_optimized.add(MaxPool2D(pool_size=(2,2)))

model_optimized.add(Conv2D(filters=8, kernel_size=(5,5), activation='relu', kernel_regularizer=l2(0.01)))
model_optimized.add(MaxPool2D(pool_size=(2,2)))

model_optimized.add(Flatten())
model_optimized.add(Dense(128, activation="relu", kernel_regularizer=l1(0.01)))
model_optimized.add(Dropout(0.5))  # Dropout layer
model_optimized.add(Dense(64, activation="relu", kernel_regularizer=l1(0.01)))
model_optimized.add(Dense(len(CLASS_NAME), activation="softmax"))



# Compile the model with the same loss function and optimizer
model_optimized.compile(loss='categorical_crossentropy', optimizer=RMSprop(0.0001), metrics=['accuracy'])

In [ ]:
# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# Train the model with the same parameters as before
history_optimized = model_optimized.fit(datagen.flow(X_train, Y_train, batch_size=84), epochs=20, validation_data=(X_val, Y_val), callbacks=[early_stopping])

In [ ]:
# Plot the training history for the optimized model
plt.figure(figsize=(12,  5))
plt.plot(history_optimized.history['accuracy'], color='r')
plt.plot(history_optimized.history['val_accuracy'], color='b')
plt.title('Optimized Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
# Make predictions using the optimized model
Y_pred_optimized = model_optimized.predict(X_test)
score_optimized = model_optimized.evaluate(X_test, Y_test)
print('Accuracy over the test set for optimized model: \n ', round((score_optimized[1]*100),  2), '%')

In [ ]:
# Save the optimized model
model_optimized.save('saved_models/model_optimized.h5')

In [ ]:
# Plotting an image from the test set
plt.imshow(X_test[1,:,:,:])
plt.show()

# Finding the index of the maximum value from the prediction list
# and comparing the original value with the predicted value
print("Original breed: ", labels['breed'][np.argmax(Y_test[1])])
print("Predicted breed: ", labels['breed'][np.argmax(Y_pred_optimized[1])])


# Conclusion
In this project, we explored the task of classifying dog breeds using convolutional neural networks (CNNs). We implemented two models: a baseline CNN model and an optimized CNN model with regularization techniques.

# Results:
The baseline model achieved an accuracy of approximately 33.9% on the test set, indicating moderate performance.
The optimized model, despite incorporating regularization techniques, achieved a lower accuracy of around 18.64% on the test set. This unexpected result suggests that further experimentation and tuning of hyperparameters may be necessary to improve model performance.
# Observations:
Both models exhibited signs of overfitting, as evidenced by the significant difference in performance between the training and validation/test sets.
The optimization techniques applied did not yield the expected improvements in model performance. This could be due to suboptimal hyperparameters or other factors influencing the training process.
# Future Work:
Further experimentation with hyperparameters, such as learning rate, batch size, and regularization strength, may be necessary to improve model performance.
Exploring more advanced optimization techniques, such as dropout and batch normalization, could potentially yield better results.
Conducting error analysis to identify the types of mistakes made by the models and potential avenues for improvement.
In conclusion, while the implemented models provide a foundation for dog breed classification, there is room for improvement in terms of model performance and generalization to unseen data. Continued experimentation and refinement of the models are necessary to achieve better results.